In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as spo
from utilities import Stocks, Portfel

In [2]:
start_date = '2019-01-01'
end_date = '2019-07-17'
dates = pd.date_range(start_date, end_date)
symbols = ['SPY', 'GOOG', 'XOM', 'GLD', 'IBM', 'AAPL']
allocs = [0.3, 0.1, 0.1, 0.1, 0.1, 0.3]
aum = 100000

In [3]:
portfel = Portfel(symbols, dates, allocs, aum)

In [6]:
# Эти функции позволяют оптимально аллоцировать соотношение ценных бумаг в портфеле

def f(allocs):
    # Если максим что-то другое меняем тут
    portfel.allocs = allocs
    return -portfel.returns(True)

# Вспомогательная функция нужна для оптимизатора
def bound_returner(allocs):
    bound = []
    z = len(allocs)
    for i in range(z):
        bound.append((0, 1))
    return bound

# Вспомогательная функция, нужна для определения условий оптимизации
def con(t):
    return t.sum() - 1

min_result = spo.minimize(f,
                          portfel.allocs,
                          method='SLSQP',
                          options={'disp': True},
                          bounds=bound_returner(portfel.allocs),
                          constraints={'type':'eq', 'fun': con})

# Оптимизатор не может обеспечить глобальную сходимость,
# однако если повторить его работу много раз с разной начальной инициализацией весов
# то можно попытаться добиться хорошего результата.

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.282827424629044
            Iterations: 1
            Function evaluations: 8
            Gradient evaluations: 1


In [7]:
# Посмотрим на оптимизированные данные
portfel.allocs = list(min_result.x)
returns_optimized = portfel.returns(accumulated=True)
print(returns_optimized)

# Посмотрим на оптимизированные веса
weights = list(min_result.x)
print(np.array(weights))

0.282827424629044
[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.14326478e-16
 0.00000000e+00 1.00000000e+00]
